In [23]:
import pandas as pd
import numpy as np

In [24]:
df = pd.read_csv('../scraping/cleansed_data/players_20230425190937.csv', sep=';')

# Erstelle eine Kopie des Dataframes
df_copy = df.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18335 entries, 0 to 18334
Data columns (total 48 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   LeagueCountry             18335 non-null  object 
 1   League                    18238 non-null  object 
 2   NationalLeagueLevel       18234 non-null  object 
 3   Club                      18331 non-null  object 
 4   No                        17905 non-null  float64
 5   Name                      18331 non-null  object 
 6   Value                     17899 non-null  float64
 7   ValueLastUpdate           17899 non-null  object 
 8   HighestValue              18004 non-null  float64
 9   HighestValueDate          18004 non-null  object 
 10  ValueCurrency             17899 non-null  object 
 11  Age                       18328 non-null  float64
 12  Height                    17052 non-null  float64
 13  Nationality               18331 non-null  object 
 14  Foot  

## Entfernen von Spalten und Zeilen

In [25]:
# Entferne Source, ScrapingTime, Instagram, Value-Dates & -Currency
df = df.drop(['Source', 'ScrapingTime', 'Instagram', 'ValueLastUpdate', 'HighestValueDate', 'ValueCurrency'], axis=1)

In [26]:
# Lösche alle Spieler ohne Value
df = df[df['Value'].notnull()]

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17899 entries, 0 to 18334
Data columns (total 42 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   LeagueCountry             17899 non-null  object 
 1   League                    17899 non-null  object 
 2   NationalLeagueLevel       17899 non-null  object 
 3   Club                      17899 non-null  object 
 4   No                        17590 non-null  float64
 5   Name                      17899 non-null  object 
 6   Value                     17899 non-null  float64
 7   HighestValue              17899 non-null  float64
 8   Age                       17896 non-null  float64
 9   Height                    16809 non-null  float64
 10  Nationality               17899 non-null  object 
 11  Foot                      16201 non-null  object 
 12  Position                  17899 non-null  object 
 13  Consultancy               14580 non-null  object 
 14  Suppli

## Datentypen anpassen

In [28]:
# 'ClubSince' und 'ContractUntil' als Datumsangaben einlesen
df['ClubSince'] = pd.to_datetime(df['ClubSince'])
df['ContractUntil'] = pd.to_datetime(df['ContractUntil'])

In [29]:
# 'ClubSince' und 'ContractUntil' als float Werte umwandeln (Differenz in Tagen ab Scrapingtag 11.04.2023, da dann auch die Values ermittelt wurden)
df['ClubSince'] = ((pd.to_datetime('2023-04-11') - df['ClubSince']) / np.timedelta64(1, 'D')).astype(float)
df['ContractUntil'] = ((df['ContractUntil'] - pd.to_datetime('2023-04-11')) / np.timedelta64(1, 'D')).astype(float)

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17899 entries, 0 to 18334
Data columns (total 42 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   LeagueCountry             17899 non-null  object 
 1   League                    17899 non-null  object 
 2   NationalLeagueLevel       17899 non-null  object 
 3   Club                      17899 non-null  object 
 4   No                        17590 non-null  float64
 5   Name                      17899 non-null  object 
 6   Value                     17899 non-null  float64
 7   HighestValue              17899 non-null  float64
 8   Age                       17896 non-null  float64
 9   Height                    16809 non-null  float64
 10  Nationality               17899 non-null  object 
 11  Foot                      16201 non-null  object 
 12  Position                  17899 non-null  object 
 13  Consultancy               14580 non-null  object 
 14  Suppli

## Entferne Duplikate

In [31]:
# Doppelte Spieler aus dem Dataframe entfernen
df = df.drop_duplicates(subset=['Name', 'Age', 'Nationality', 'Club'], keep='first')

## Fehlende Werte ersetzen (Objekte und logische Werte)

In [32]:
# Erstelle ein Dictionary mit den Positionen und ihren entsprechenden Kategorien
pos_dict = {'Torwart': 'Torwart',
            'Innenverteidiger': 'Abwehr',
            'Linker Verteidiger': 'Abwehr',
            'Rechter Verteidiger': 'Abwehr',
            'Defensives Mittelfeld': 'Mittelfeld',
            'Zentrales Mittelfeld': 'Mittelfeld',
            'Linkes Mittelfeld': 'Mittelfeld',
            'Offensives Mittelfeld': 'Mittelfeld',
            'Linksaußen': 'Sturm',
            'Rechtsaußen': 'Sturm',
            'Mittelstürmer': 'Sturm',
            'Rechtes Mittelfeld': 'Mittelfeld',
            'Hängende Spitze': 'Mittelfeld',
            'Mittelfeld': 'Mittelfeld',
            'Abwehr': 'Abwehr',
            'Sturm': 'Sturm'}

# Erstelle eine neue Spalte mit der Positionskategorie
df['PositionCategory'] = df['Position'].map(pos_dict)

In [33]:
# Neue Spalte 'trending' erstellen
df['Trending'] = df.apply(lambda row: 1 if row['Value'] == row['HighestValue'] else 0, axis=1)

In [34]:
df.to_csv('../data/df_clean.csv', index=False)

## Nicht verwendeter Code

In [35]:
df['InternationalTeam'].fillna('kein_internationales_Team', inplace=True)

In [36]:
df.loc[df['InternationalTeam'] == 'kein_internationales_Team', ['InternationalGames', 'InternationalGoals']] = 0

In [37]:
# Setze die Torwart-spezifischen Werte auf 0, wenn die Position des Spielers keine Torwart-Position ist und der jeweilige Wert NaN ist
df.loc[(df['Position'] != 'Torwart') & (df['TwPenaltySavesQuote'].isna()), 'TwPenaltySavesQuote'] = 0
df.loc[(df['Position'] != 'Torwart') & (df['TwGoalsConceded'].isna()), 'TwGoalsConceded'] = 0
df.loc[(df['Position'] != 'Torwart') & (df['TwCleanSheets'].isna()), 'TwCleanSheets'] = 0

In [38]:
# Gruppieren Sie das DataFrame nach Position und berechnen Sie den Modus der Spieler-Nummern für jede Position
mode_no_by_position = df.groupby('Position')['No'].agg(lambda x: x.mode()[0])

# Funktion zum Imputieren von fehlenden Spieler-Nummern (No) basierend auf der Position
def impute_no_by_position(row):
    if pd.isnull(row['No']):
        return mode_no_by_position[row['Position']]
    else:
        return row['No']

# Anwenden der Funktion auf das gesamte DataFrame, um fehlende Spieler-Nummern zu imputieren
df['No'] = df.apply(impute_no_by_position, axis=1)

In [39]:
# Setze NaN-Werte in Karten-Werten auf 0
df['YellowCards'].fillna(0, inplace=True)
df['YellowRedCards'].fillna(0, inplace=True)
df['RedCards'].fillna(0, inplace=True)

In [40]:
# Objekte haben keine fehlenden Werte mehr
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17803 entries, 0 to 18334
Data columns (total 44 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   LeagueCountry             17803 non-null  object 
 1   League                    17803 non-null  object 
 2   NationalLeagueLevel       17803 non-null  object 
 3   Club                      17803 non-null  object 
 4   No                        17803 non-null  float64
 5   Name                      17803 non-null  object 
 6   Value                     17803 non-null  float64
 7   HighestValue              17803 non-null  float64
 8   Age                       17800 non-null  float64
 9   Height                    16713 non-null  float64
 10  Nationality               17803 non-null  object 
 11  Foot                      16118 non-null  object 
 12  Position                  17803 non-null  object 
 13  Consultancy               14496 non-null  object 
 14  Suppli

In [41]:
# Fehlende Werte in Prozent pro Variable
print(df.isnull().sum() / len(df) * 100)

LeagueCountry                0.000000
League                       0.000000
NationalLeagueLevel          0.000000
Club                         0.000000
No                           0.000000
Name                         0.000000
Value                        0.000000
HighestValue                 0.000000
Age                          0.016851
Height                       6.122564
Nationality                  0.000000
Foot                         9.464697
Position                     0.000000
Consultancy                 18.575521
Supplier                    84.631804
ClubSince                    0.196596
ContractUntil                6.027074
Injury                       0.000000
InternationalTeam            0.000000
ActiveInternational          0.000000
FormerInternational          0.000000
InternationalGames           0.000000
InternationalGoals           0.000000
StartingElevenQuote         11.402573
MinutesQuote                11.402573
TwPenaltySavesQuote          1.628939
FsGoalPartic

In [42]:
# Manuelles Mapping erstellen
league_level_mapping = {
    "1.Liga": 1,
    "2.Liga": 2,
    "3.Liga": 3,
    "4.Liga": 4,
    "5.Liga": 5,
    "Reserveliga": 6,
    "Jugendliga": 6,
    "Playoff/Relegation": 7
}

# Ersetzen Sie die Werte in der Spalte "NationalLeagueLevel" basierend auf dem manuellen Mapping
df['NationalLeagueLevelCategory'] = df['NationalLeagueLevel'].map(league_level_mapping)

In [43]:
null_values = df['Minutes'].isnull().groupby([df['LeagueCountry'], df['League']]).sum().to_frame()
size_values = df.groupby(['LeagueCountry', 'League']).size()

null_values['Players'] = size_values
null_values = null_values.rename(columns={'Minutes': 'NullValues'})
null_values['NullValuesRelative'] = null_values['NullValues'] / null_values['Players'] * 100

null_values = null_values[null_values['Players'] > 20]  # exclude minor leagues
null_values = null_values.sort_values('NullValuesRelative', ascending=False)

print(null_values)

                                     NullValues  Players  NullValuesRelative
LeagueCountry League                                                        
Türkei        Süper Lig                     520      520          100.000000
Niederlande   Eredivisie                    481      481          100.000000
Griechenland  Super League 2                521      725           71.862069
Norwegen      Eliteserien                   279      414           67.391304
Schottland    Championship                  153      237           64.556962
Schweiz       Promotion League              250      394           63.451777
Spanien       LaLiga2                       300      531           56.497175
Schweden      Allsvenskan                   217      389           55.784062
England       League One                    333      624           53.365385
Tschechien    Fortuna Liga                  160      405           39.506173
Dänemark      Superligaen                   108      289           37.370242

In [44]:
# Ersetze NaN-Werte mit 'Unbekannt'
df['Nationality'].fillna('unbekannt', inplace=True)
df['Foot'].fillna('unbekannt', inplace=True)
df['Position'].fillna('unbekannt', inplace=True)
df['Supplier'].fillna('unbekannt', inplace=True)
df['Consultancy'].fillna('unbekannt', inplace=True)